In [ ]:
#importing libraries
import os
import sys
import cv2 as cv
import numpy as np
import plotly.io as pio
import plotly.graph_objs as go


from PIL import Image
from skimage import color
from plotly import subplots
from sklearn.model_selection import train_test_split


#setting the rederer as colab
pio.renderers.default = "colab"

# **LOADING DATASET**

In [ ]:
#Loading data and reducing size to 64 x 64 pixels
IMG_DIR = '/content/sample_data/images'
X = []
X_flat = []
count = 1
size = 32
total = 919
print("Loading...")
for img in os.listdir(IMG_DIR):
    if count == total + 1:
        break
    sys.stdout.write("\r" + str(count) + " / " + str(total))
    sys.stdout.flush()
    img_array = cv.imread(os.path.join(IMG_DIR, img), cv.IMREAD_GRAYSCALE)
    img_pil = Image.fromarray(img_array)
    img_32x32 = np.array(img_pil.resize((size, size), Image.ANTIALIAS))
    X.append(img_32x32)
    img_array = img_32x32.flatten()
    X_flat.append(img_array)
    count += 1
print()
print("Done!")

Loading...
39 / 919

<ipython-input-17-ef7372d21d7b>:16: DeprecationWarning:

ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.



919 / 919
Done!


# **VISUALIZING**

In [ ]:
#visualizing some images
size = 4
count = 0
fig = subplots.make_subplots(rows = size, cols = size,
                 vertical_spacing = 0.06, horizontal_spacing = 0.02)
for row in range(size):
  for col in range(size):
    fig.add_trace(go.Image(z = color.gray2rgb(X[count])),
                  row = row + 1, col = col + 1)
    count += 1
fig["layout"].update(title = "Preview Images", template = "plotly_dark", height = 900)
fig.show()

In [ ]:
#converting X_flat to numpy array
X_flat = np.asarray(X_flat)
X_flat.shape

(919, 1024)

In [ ]:
#Test-Train split
X_train, X_test = train_test_split(X_flat,
                           test_size = 0.5, random_state = 69)
X_train = X_train.T
X_test = X_test.T
X_test = X_test[:,:-1]
print(X_train.shape)
print(X_test.shape)

(1024, 459)
(1024, 459)


# **STANDARDIZATION**

In [ ]:
#function for standardizing image
def Standardize(X):
    #calcualte the mean of each column mu
    mu = np.mean(X, axis = 0)

    #Substract the mean from X
    X = X - mu

    #calcualte the standard deviation of each column std
    std = np.std(X, axis = 0)

    #handleing zero standard deviation
    std_filled = std.copy()
    std_filled[std == 0] = 1.0

    #calculate standardized X called Xbar
    Xbar = (X-mu) / std_filled

    return Xbar, mu, std

# **EIGENVALUES AND EIGENVECTORS AND THEIR SORTING**

In [ ]:
#function for calcualting eigen values and eigen vectors
def eig(S):
    #calculate the eigen values and eigen vectors
    eig_val, eig_vec = np.linalg.eigh(S)

    #sorting them in decrasing order
    sorted_eig  = np.argsort(-eig_val)
    eig_val = eig_val[sorted_eig]
    eig_vec = eig_vec[:, sorted_eig]

    return (eig_val, eig_vec)

# **PROJECTION MATRIX**

In [ ]:
#function for projection matrix
def projection_matrix(B):
    #calculate the projection matrix P
    P = B @ B.T

    return P

# **PCA FUNCTION**

In [ ]:
#implementing PCA
def PCA(X, num_components):
    #calculate the data covariance matrix S
    S = np.cov(X)

    #now find eigenvalues and corresponding eigenvectors for S
    #by implementing eig().
    eig_vals, eig_vecs = eig(S)

    #select eigen vectors
    U = eig_vecs[:, range(num_components)]

    #reconstruct the images from the lowerdimensional representation
    #to do this, we first need to find the projection_matrix
    #(which you implemented earlier)
    #which projects our input data onto the vector space spanned
    #by the eigenvectors
    P = projection_matrix(U) # projection matrix

    return P

In [ ]:
Xbar_train, mu_train, std_train = Standardize(X_train)
Xbar_test, mu_test, std_test = Standardize(X_test)

In [ ]:
#function for mean square error
def mse(predict, actual):
    return np.square(predict - actual).sum(axis = 1).mean()

In [ ]:
#calculating loss and reconstructing images
loss = []
reconstructions = []
max_components = len(X_train.T)
print("Processing...")
animation = np.arange(1, max_components + 1, 1)
for num_component in range(1, max_components + 1):
    sys.stdout.write("\r" + str(animation[num_component - 1]) + \
                      " / " + str(max_components))
    sys.stdout.flush()
    projection = PCA(Xbar_train.T, num_component)
    reconst = Xbar_test @ projection
    error = mse(reconst, Xbar_test)
    reconstructions.append(reconst)
    loss.append((num_component, error))
print()
print("Done!")

Processing...
459 / 459
Done!


In [ ]:
#visualizing mse vs number of principal components
loss=np.array(loss)
trace = go.Scatter(x = loss[:, 0], y = loss[:, 1])
data = [trace]
fig = go.Figure(data)
fig.update_layout(title = "MSE vs number of principal components",
                  xaxis_title = "Number of principal components",
                  yaxis_title = "MSE", template = "plotly_dark")
fig.show()

# **UNSTANDARDIZE IMAGES TO GET CORRECT PIXEL VALUES**

Here, I had got the results previously, but later when I was running again before the fnal submission, my GPU runtime got over, so I don't have the final results here. Without GPU I could only run till above code

In [ ]:
#unstandardizing the reconstructed images
reconstructions = np.asarray(reconstructions)
reconstructions = reconstructions * std_test + mu_test
loss = np.asarray(loss)
print("Done")

Done


In [ ]:
#plotting the reconstructed images
col = 5
row = 2
size = 10
skip = 50
images = 2


component = 0
titles = []
for loop in range(size):
  titles.append(str(component) + " components")
  component += skip


figs = [0]*images
for num in range(images):
  figs[num] = subplots.make_subplots(rows = row, cols = col, subplot_titles = titles)
  component = 0
  for r in range(row):
    for c in range(col):
      figs[num].add_trace(go.Image(
             z = color.gray2rgb(
             reconstructions[component, : , num].reshape(32, 32))),
             row = r + 1, col = c + 1)
      component += skip
  figs[num]["layout"].update(title = "Recontructed Images",
                             template = "plotly_dark")


for num in range(images):
  figs[num].show()